In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import mlflow
from azureml.core import Workspace

In [59]:
ws = Workspace.from_config('../../ML/')
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [70]:
ws.compute_targets

{}

In [60]:
from azureml.core import Experiment
experiment_name = 'co_Day1DangerAboveTreeline_small'
exp = Experiment(workspace=ws, name=experiment_name)

In [61]:
from azureml.core import Environment

myenv = Environment("user-managed-env")
myenv.python.user_managed_dependencies = True

In [62]:
from azureml.train.hyperdrive import BayesianParameterSampling
from azureml.train.hyperdrive import uniform, choice

In [63]:
param_sampling = BayesianParameterSampling( 
    {
        "model_index": choice(0,1),
        "hidden": choice(50,100,200),
        "dropout": choice(0, .5, .9),
        "fc_dropout": choice(0, .5, .9),
        "layers": choice(3,6,9),
        "bidirectional": choice(0, 1),
    }
)

In [64]:
?ScriptRunConfig

In [65]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target='local',
                      environment=myenv)

In [66]:
from azureml.train.hyperdrive import HyperDriveConfig
from azureml.train.hyperdrive import PrimaryMetricGoal
hd_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=param_sampling,                             
                             primary_metric_name="Test Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=25,
                             max_concurrent_runs=1)

In [67]:
src.run_config.target

'local'

In [68]:
hd_config.run_config.run_config.target

'local'

In [69]:

hyperdrive_run = exp.submit(hd_config)

ComputeTargetException: ComputeTargetException:
	Message: ComputeTargetNotFound: Compute Target with name local not found in provided workspace
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "ComputeTargetNotFound: Compute Target with name local not found in provided workspace"
    }
}

In [38]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: co_Day1DangerAboveTreeline_small_1610830126_38905b3a
Web View: https://ml.azure.com/experiments/co_Day1DangerAboveTreeline_small/runs/co_Day1DangerAboveTreeline_small_1610830126_38905b3a?wsid=/subscriptions/65e24061-f14f-4eaa-bc0b-93690b981022/resourcegroups/oapmlresourcegroup/workspaces/oapmlworkspace

Streaming azureml-logs/70_driver_log.txt

[2021-01-16T20:48:54.303378] Entering context manager injector.
[context_manager_injector.py] Command line Options: Namespace(inject=['ProjectPythonPath:context_managers.ProjectPythonPath', 'RunHistory:context_managers.RunHistory', 'TrackUserError:context_managers.TrackUserError'], invocation=['train.py'])
Script type = None
Starting the daemon thread to refresh tokens in background for process with pid = 8595
Entering Run History Context Manager.
[2021-01-16T20:48:57.956144] Current directory: /tmp/azureml_runs/co_Day1DangerAboveTreeline_small_1610830126_38905b3a
[2021-01-16T20:48:57.956218] Preparing to call script [train.py] with argum

{'runId': 'co_Day1DangerAboveTreeline_small_1610830126_38905b3a',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-01-16T20:48:53.421827Z',
 'endTimeUtc': '2021-01-16T20:50:24.313485Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '10dca0ac-3075-4b74-b79b-6f5e9597d471',
  'azureml.git.repository_uri': 'https://github.com/scottcha/OpenAvalancheProject.git',
  'mlflow.source.git.repoURL': 'https://github.com/scottcha/OpenAvalancheProject.git',
  'azureml.git.branch': 'update_ml',
  'mlflow.source.git.branch': 'update_ml',
  'azureml.git.commit': '3946019d9d8bbadbb88935350614b5487405e087',
  'mlflow.source.git.commit': '3946019d9d8bbadbb88935350614b5487405e087',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 